# Notebook for both CNN's and Convolution + Machine Learning

## Importing Libraries

In [ ]:
import numpy as np
import cv2
import albumentations as A
import torch
from pathlib import Path
import json
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Mounted at /content/drive


## Dataset Preparation

This code gets the dataset.zip from the TKPR221 folder in Google Drive and unzips it to the colab workspace, so that it is much faster when loading the data to the model.

Delete comment signs (''') to make this block work.

In [ ]:
import shutil
from pathlib import Path
def extract_drive_dataset(
    zip_path='/content/drive/MyDrive/TKPR221/traffic_signs_dataset.zip',
    extract_path='/content/dataset'
):
    """
    Copy ZIP from Drive to Colab and extract it
    """
    zip_path = Path(zip_path)
    if not zip_path.exists():
        raise FileNotFoundError(f"ZIP file not found at: {zip_path}")

    # Create temporary directory for ZIP
    temp_dir = Path('/content/temp')
    temp_dir.mkdir(exist_ok=True)
    temp_zip = temp_dir / 'dataset.zip'

    # Copy ZIP to Colab
    print(f"\nCopying ZIP from Drive to Colab...")
    print(f"Source: {zip_path}")
    print(f"Destination: {temp_zip}")
    shutil.copy2(zip_path, temp_zip)

    # Verify copy
    if temp_zip.exists():
        zip_size = temp_zip.stat().st_size / (1024 * 1024)  # Size in MB
        print(f"ZIP copied successfully! Size: {zip_size:.2f} MB")
    else:
        raise RuntimeError("Failed to copy ZIP file")

    # Create extraction directory
    extract_dir = Path(extract_path)
    extract_dir.mkdir(parents=True, exist_ok=True)

    # Extract ZIP
    print(f"\nExtracting ZIP to: {extract_dir}")
    print("This might take a while...")
    shutil.unpack_archive(str(temp_zip), str(extract_dir))

    # Clean up temporary ZIP
    print("\nCleaning up temporary files...")
    temp_zip.unlink()
    temp_dir.rmdir()

    # Verify extraction
    crops_dir = extract_dir / 'crops'
    csv_file = crops_dir / 'dataset.csv'

    print("\nVerifying extracted files:")
    print(f"Crops directory exists: {crops_dir.exists()}")
    print(f"Dataset CSV exists: {csv_file.exists()}")

    if crops_dir.exists():
        batch_folders = list(crops_dir.glob('batch_*'))
        print(f"Number of batch folders found: {len(batch_folders)}")

        # Print some statistics
        total_images = sum(len(list(folder.glob('*.jpg'))) for folder in batch_folders)
        print(f"Total number of images found: {total_images}")

        # Print folder structure
        print("\nFolder structure:")
        print(f"└── {crops_dir.name}/")
        print(f"    ├── dataset.csv")
        for i, batch in enumerate(sorted(batch_folders)):
            is_last = i == len(batch_folders) - 1
            prefix = "    └── " if is_last else "    ├── "
            n_images = len(list(batch.glob('*.jpg')))
            print(f"{prefix}{batch.name}/ ({n_images} images)")

    return str(crops_dir)

# Usage
try:
    dataset_dir = extract_drive_dataset()
    print(f"\nDataset extracted successfully to: {dataset_dir}")
except Exception as e:
    print(f"Error: {str(e)}")


Copying ZIP from Drive to Colab...
Source: /content/drive/MyDrive/TKPR221/traffic_signs_dataset.zip
Destination: /content/temp/dataset.zip
ZIP copied successfully! Size: 126.81 MB

Extracting ZIP to: /content/dataset
This might take a while...

Cleaning up temporary files...

Verifying extracted files:
Crops directory exists: False
Dataset CSV exists: False

Dataset extracted successfully to: /content/dataset/crops


This code generates a PyTorch dataset. Custom PyTorch dataset allows us to:


1.   Batch train sklearn models.
2.   Apply augmentations to the data.
3.   Add more customizations if needed.



In [ ]:
# Custom dataset class for batch loading and transformations
class TrafficSignDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, csv, transformations=None, size=None, flatten=False, label_encoder=None,reorder=False,tensor=False):
        self.image_dir = Path(image_dir)
        self.csv = csv
        self.transformations = transformations
        self.size = size
        self.flatten = flatten
        self.reorder = reorder
        self.tensor = tensor

        if label_encoder is None:
            self.label_encoder = LabelEncoder()
            self.csv['label'] = self.label_encoder.fit_transform(self.csv['label'])
        else:
            self.label_encoder = label_encoder
            self.csv['label'] = self.label_encoder.transform(self.csv['label'])

        # Store labels for class weights
        self.labels = self.csv['label'].values

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        # Get image data and load the image
        image_data = self.csv.iloc[idx]
        image = cv2.imread(f"{self.image_dir}/{image_data['filename']}")
        # Apply transformations (using albumentations)
        if self.transformations:
            transformed = self.transformations(image=image)
            image = transformed['image']
        if self.size is not None:
            image = self.resize_with_pad(image, (self.size, self.size))
        if self.reorder:
            image = self.rearrange(image)
        if self.flatten:
            image = image.flatten()

        if self.tensor:
            image = torch.from_numpy(image).float()
            image_label = torch.tensor(image_data['label'],dtype=torch.long)
        else:
            image = image.astype(np.float32) / 255.0
            image_label = image_data['label']
        return image, image_label

    def transform(self, image):
        return self.transformations(image)


    def resize_with_pad(self,image: np.array,
                    new_shape: tuple[int, int],
                    padding_color: tuple[int, ...] = (0, 0, 0)) -> np.array:

        ## COPIED FROM : https://gist.github.com/IdeaKing/11cf5e146d23c5bb219ba3508cca89ec
        """Maintains aspect ratio and resizes with padding.
        Params:
            image: Image to be resized.
            new_shape: Expected (width, height) of new image.
            padding_color: Tuple in BGR of padding color
        Returns:
            image: Resized image with padding
        """
        original_shape = (image.shape[1], image.shape[0])
        ratio = float(max(new_shape))/max(original_shape)
        new_size = tuple([int(x*ratio) for x in original_shape])
        image = cv2.resize(image, new_size)
        delta_w = new_shape[0] - new_size[0]
        delta_h = new_shape[1] - new_size[1]
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)
        image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=padding_color)
        return image

    # Some helper functions for partial_fit in sklearn
    def get_label_encoded_classes(self):
        classes = self.label_encoder.classes_
        encoded_classes = self.label_encoder.transform(classes)
        return encoded_classes

    def rearrange(self,image):
        return np.moveaxis(image, [0,1,2], [2,1,0])


## Modeling

### CNN MODEL:



In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, n_classes, return_convolution=False):
        super().__init__()
        self.return_convolution = return_convolution
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.pooling = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, n_classes)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.pooling(F.leaky_relu(self.batchnorm1(self.conv1(x))))
        x = self.dropout(x)
        x = self.pooling(F.leaky_relu(self.batchnorm2(self.conv2(x))))
        x = self.dropout(x)
        x = self.pooling(F.leaky_relu(self.batchnorm3(self.conv3(x))))
        x = self.dropout(x)

        x = x.view(x.size(0), -1)  # Flatten

        if self.return_convolution:
            return x

        x = self.dropout(F.leaky_relu(self.fc1(x)))
        x = self.dropout(F.leaky_relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [ ]:
import torchvision.models as models
from torchvision import transforms

class TrafficSignResNet(nn.Module):
    def __init__(self, num_classes, model_name='resnet50', pretrained=True):
        super().__init__()
        if model_name == 'resnet50':
            self.backbone = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        # Freeze early layers
        for param in list(self.backbone.parameters())[:-30]:
            param.requires_grad = False
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

In [ ]:
!pip install neptune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.4 MB/s eta 0:00:00
  Created wheel for bravado-core: filename=bravado_core-6.1.1-py2.py3-none-any.whl size=67675 sha256=6a01a2a5406651dccc2f83cd713777828ed4af4cd6d9fc8ba723083058e73f9f
  Stored in directory: /root/.cache/pip/wheels/42/35/4a/44ec4c358db21a5d63ed4e40f0f0012a438106f220bce4ccba
Successfully built bravado-core


In [ ]:
!rm -rf /content/dataset/traffic_signs_dataset/synthetic_images

In [ ]:
import pandas as pd
import numpy as np
import cv2
from pathlib import Path
from imblearn.over_sampling import SMOTE

# Set these paths from google drive
# You need to set the TKPR folder as your favourite to see in the colab
crops_dir = "/content/dataset/traffic_signs_dataset/"
dataset_csv = "/content/dataset/traffic_signs_dataset/dataset.csv"
df = pd.read_csv(dataset_csv)

# Get class counts and find biggest classes
class_counts = df['label'].value_counts()
print("Original class distribution:")
print(class_counts)

# Select 1200 samples from each of the biggest classes
samples_per_class = 1200
df_filtered = pd.DataFrame()
for label in class_counts.index:
    class_samples = df[df['label'] == label]
    if len(class_samples) > samples_per_class:
        df_filtered = pd.concat([df_filtered, class_samples.sample(n=samples_per_class, random_state=42)])
    else:
        df_filtered = pd.concat([df_filtered, class_samples])

# Create directory for synthetic images
synthetic_dir = Path(crops_dir) / "synthetic_images"
synthetic_dir.mkdir(exist_ok=True)

image_data = []
paths = []
labels = []

for _, row in df_filtered.iterrows():
    img_path = Path(crops_dir) / row['filename']
    img = cv2.imread(str(img_path))
    img = cv2.resize(img, (32, 32))
    image_data.append(img.flatten())
    paths.append(row['filename'])
    labels.append(row['label'])

X = np.array(image_data)
y = np.array(labels)

# SMOTE
min_samples = 6
valid_labels = []
for label in np.unique(y):
    if np.sum(y == label) >= min_samples:
        valid_labels.append(label)

# Filter data to only include classes with sufficient samples
mask = np.isin(y, valid_labels)
X = X[mask]
y = y[mask]
filtered_paths = np.array(paths)[mask]

smote = SMOTE(sampling_strategy={label: samples_per_class for label in valid_labels}, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Save original and synthetic images
all_new_paths = []
all_new_labels = []

for i, (img_array, label) in enumerate(zip(X_resampled, y_resampled)):
    if i < len(filtered_paths):  # Original image
        all_new_paths.append(filtered_paths[i])
    else:  # Synthetic image
        img = img_array.reshape(32, 32, 3).astype(np.uint8)
        synthetic_path = f"synthetic_images/synthetic_{label}_{i}.jpg"
        cv2.imwrite(str(Path(crops_dir) / synthetic_path), img)
        all_new_paths.append(synthetic_path)
    all_new_labels.append(label)

# Create balanced dataframe
df_balanced = pd.DataFrame({
    'filename': all_new_paths,
    'label': all_new_labels
})

print("\nBalanced class distribution:")
print(df_balanced['label'].value_counts())
print(f"\nSynthetic images saved to: {synthetic_dir}")

# Use the balanced dataframe for further processing
df_filtered = df_balanced.copy()


Original class distribution:
label
other-sign                                               39121
regulatory--yield--g1                                      827
information--pedestrians-crossing--g1                      685
regulatory--no-entry--g1                                   600
complementary--chevron-right--g1                           531
                                                         ...  
warning--restricted-zone--g1                                 2
regulatory--turning-vehicles-yield-to-pedestrians--g1        2
regulatory--roundabout--g2                                   2
regulatory--end-of-maximum-speed-limit-70--g1                2
regulatory--dual-lanes-turn-right-or-straight--g1            1
Name: count, Length: 399, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(



Balanced class distribution:
label
regulatory--end-of-maximum-speed-limit-30--g2    1200
other-sign                                       1200
regulatory--yield--g1                            1200
information--pedestrians-crossing--g1            1200
regulatory--no-entry--g1                         1200
                                                 ... 
regulatory--one-way-straight--g1                 1200
regulatory--keep-right--g4                       1200
regulatory--no-parking--g5                       1200
regulatory--height-limit--g1                     1200
warning--road-bump--g2                           1200
Name: count, Length: 368, dtype: int64

Synthetic images saved to: /content/dataset/traffic_signs_dataset/synthetic_images


Creating the dataset instance and dataloaders.

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

# Split into train and test sets (80-20 split)
train_df, test_df = train_test_split(df_filtered, test_size=0.2, stratify=df_filtered['label'])

# Further split training data into train and validation (80-20 split of training data)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'])

print("Dataset splits:")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Testing samples: {len(test_df)}")

# Verify class distribution in splits
print("\nClass distribution in training set:")
print(train_df['label'].value_counts())
print("\nClass distribution in validation set:")
print(val_df['label'].value_counts())
print("\nClass distribution in test set:")
print(test_df['label'].value_counts())


Dataset splits:
Training samples: 282624
Validation samples: 70656
Testing samples: 88320

Class distribution in training set:
label
warning--roadworks--g1                            768
regulatory--keep-left--g1                         768
regulatory--no-parking-or-no-stopping--g1         768
warning--turn-left--g1                            768
regulatory--maximum-speed-limit-65--g2            768
                                                 ... 
regulatory--dual-lanes-turn-left-no-u-turn--g1    768
warning--double-curve-first-right--g2             768
information--pedestrians-crossing--g1             768
complementary--turn-right--g2                     768
regulatory--end-of-buses-only--g1                 768
Name: count, Length: 368, dtype: int64

Class distribution in validation set:
label
regulatory--no-left-turn--g2                192
regulatory--maximum-speed-limit-45--g3      192
warning--flaggers-in-road--g1               192
regulatory--go-straight-or-turn-left--g1    1

In [ ]:
from torch.utils.data import random_split
import neptune
from sklearn.linear_model import SGDClassifier
import time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import torch
import torch.nn as nn
import torch.optim as optim

# Hyperparameters
n_epochs = 10
batch_size = 128
train_size = 0.8
val_size = 0.1
test_size = 0.1
image_size = 224

train_transform = A.Compose([
    A.RandomBrightnessContrast(p=0.2),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, p=0.3),
    A.Normalize()
])

val_transform = A.Compose([
    A.Normalize()
])


train_dataset = TrafficSignDataset(
    crops_dir,
    train_df,
    transformations=train_transform,
    flatten=False,
    reorder=True,
    tensor=True,
    size=224
)

val_dataset = TrafficSignDataset(
    crops_dir,
    val_df,
    transformations=val_transform,
    flatten=False,
    reorder=True,
    tensor=True,
    size=224,
    label_encoder=train_dataset.label_encoder
)

test_dataset = TrafficSignDataset(
    crops_dir,
    test_df,
    transformations=val_transform,
    flatten=False,
    reorder=True,
    tensor=True,
    size=224,
    label_encoder=train_dataset.label_encoder
)


Getting the device (for training on gpu), and creating loss, optimizer and model objects.

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

from sklearn.utils.class_weight import compute_class_weight

# Create data loaders
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=4
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    num_workers=4
)

n_classes = len(train_dataset.label_encoder.classes_)
model = TrafficSignResNet(n_classes).to(device)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0.0001
)
criterion = nn.CrossEntropyLoss().to(device)



Using device: cuda


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 224MB/s]


Initializing Neptune run for the experiment tracking.

You can change the run structure.

In [ ]:
run_name = f"CNN_Synthetic{time.strftime('%Y%m%d_%H%M%S')}"
run = neptune.init_run(
    project="muguryalcin/TKPR221",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhZmVmNTk2YS1iZDFmLTQ5NDAtODBkZi01NjFiMWQxOTg0NWYifQ==",
    name=run_name,
    description="CNN model for traffic sign classification",
    tags=["CNN", "Custom Architecture", "Synthetic Data"]
)

# Log the hyperparameters
run["hyperparameters"] = {
    "n_epochs": n_epochs,
    # Model Architecture Parameters
    "model": {
        "type": "Custom CNN",
        "architecture": [
            "Conv2d(3, 16, 3, padding=1)",
            "MaxPool2d(2, 2)",
            "Dropout(0.2)",
            "Conv2d(16, 32, 3, padding=1)",
            "MaxPool2d(2, 2)",
            "Dropout(0.2)",
            "Linear(32 * 56 * 56, 128)",
            "Dropout(0.2)",
            "Linear(128, n_classes)"
        ]
    },
    # Optimizer Parameters
    "optimizer": {
        "type": "AdamW",
        "learning_rate": 0.001,
        "weight_decay": 0.01
    },
    # Loss Function
    "loss": {
        "type": "CrossEntropyLoss"
    },
    # Dataset Specific Parameters
    "dataset": {
        "batch_size": batch_size,
        "num_workers": 4,
        "pin_memory": True
    }
}


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/muguryalcin/TKPR221/e/TKPR-192


In [ ]:
run_name = f"ResNet_Synthetic{time.strftime('%Y%m%d_%H%M%S')}"
run = neptune.init_run(
    project="muguryalcin/TKPR221",
    api_token="",
    name=run_name,
    description="ResNet50 model for traffic sign classification",
    tags=["ResNet50", "Transfer Learning","Synthetic Data"]
)
run["hyperparameters"] = {
    "n_epochs": n_epochs,
    "model": {
        "type": "ResNet50",
        "pretrained": True,
        "backbone": "resnet50",
        "fc_layers": [
            {"in_features": 2048, "out_features": 512},
            {"in_features": 512, "out_features": 256},
            {"in_features": 256, "out_features": n_classes}
        ],
        "dropout_rates": [0.5, 0.3, 0.2]
    },
    "optimizer": {
        "type": "AdamW",
        "learning_rate": 1e-4,
        "weight_decay": 0.01
    },
    "loss": {
        "type": "CrossEntropyLoss"
    },
    "dataset": {
        "batch_size": 32,
        "num_workers": 4,
        "image_size": 224,  # ResNet standard input size
        "normalization": {
            "mean": [0.485, 0.456, 0.406],
            "std": [0.229, 0.224, 0.225]
        },
        "augmentations": {
            "train": ["Resize", "HorizontalFlip", "RandomBrightnessContrast", "Normalize"],
            "val": ["Resize", "Normalize"]
        }
    },
    "lr_scheduler": {
        "type": "ReduceLROnPlateau",
        "mode": "min",
        "factor": 0.1,
        "patience": 3
    }
}


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/muguryalcin/TKPR221/e/TKPR-193


Training loop of the CNN.

In [ ]:
# Early Stopping
best_val_loss = float('inf')
patience = 3
patience_counter = 0

# LR Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Gradient Scaler
scaler = torch.cuda.amp.GradScaler()

model_save_path = f'/content/drive/MyDrive/TKPR221/models/{run_name}.pth'

for epoch in tqdm(range(n_epochs), desc="Training epochs"):
    # Train
    model.train()
    train_loss = 0
    train_preds = []
    train_labels = []
    print(f"\nStarting epoch {epoch+1}/{n_epochs}")
    for i, (images, batch_labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}", leave=False)):
        images = images.to(device)
        batch_labels = batch_labels.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, batch_labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()

        _, batch_preds = torch.max(outputs.data, 1)
        train_preds.extend(batch_preds.cpu().numpy())
        train_labels.extend(batch_labels.cpu().numpy())

        if i % 25 == 0:
            batch_accuracy = accuracy_score(train_labels, train_preds)
            batch_f1 = f1_score(train_labels, train_preds, average='weighted')
            print(f"Batch {i}: Training loss = {loss.item():.4f}, Training accuracy = {batch_accuracy:.4f}, F1 = {batch_f1:.4f}")

    # Calculating Train Metrics
    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = accuracy_score(train_labels, train_preds)
    train_f1 = f1_score(train_labels, train_preds, average='weighted')
    train_precision = precision_score(train_labels, train_preds, average='weighted')
    train_recall = recall_score(train_labels, train_preds, average='weighted')
    # Logging Train Metrics
    run["training/loss"].append(avg_train_loss)
    run["training/accuracy"].append(train_accuracy)
    run["training/f1"].append(train_f1)
    run["training/precision"].append(train_precision)
    run["training/recall"].append(train_recall)
    # Printing Train Metrics
    print(f"\nEpoch {epoch+1} Training Metrics:")
    print(f"Loss: {avg_train_loss:.4f}")
    print(f"Accuracy: {train_accuracy:.4f}")
    print(f"F1 Score: {train_f1:.4f}")
    print(f"Precision: {train_precision:.4f}")
    print(f"Recall: {train_recall:.4f}")
    # Validation
    print("\nStarting validation...")
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for images, batch_labels in tqdm(val_loader, desc="Validation", leave=False):
            images = images.to(device)
            batch_labels = batch_labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, batch_labels)
            val_loss += loss.item()

            _, batch_preds = torch.max(outputs.data, 1)
            val_preds.extend(batch_preds.cpu().numpy())
            val_labels.extend(batch_labels.cpu().numpy())
    # Calculating Validation Metrics
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = accuracy_score(val_labels, val_preds)
    val_f1 = f1_score(val_labels, val_preds, average='weighted')
    val_precision = precision_score(val_labels, val_preds, average='weighted')
    val_recall = recall_score(val_labels, val_preds, average='weighted')
    # Logging Validation Metrics
    run["validation/loss"].append(avg_val_loss)
    run["validation/accuracy"].append(val_accuracy)
    run["validation/f1"].append(val_f1)
    run["validation/precision"].append(val_precision)
    run["validation/recall"].append(val_recall)
    # Printing Validation Metrics
    print(f"\nEpoch {epoch+1} Validation Metrics:")
    print(f"Loss: {avg_val_loss:.4f}")
    print(f"Accuracy: {val_accuracy:.4f}")
    print(f"F1 Score: {val_f1:.4f}")
    print(f"Precision: {val_precision:.4f}")
    print(f"Recall: {val_recall:.4f}")

    scheduler.step(avg_val_loss)
    # Looking for early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': avg_val_loss,
        }, model_save_path)
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Stopped {epoch + 1} epochs")
            break

# Save the trained model
model_save_path = f'/content/drive/MyDrive/TKPR221/models/{run_name}.pth'
print("Saving the model...")
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, model_save_path)

print(f"Model saved to {model_save_path}")
checkpoint = torch.load(model_save_path)
model.load_state_dict(checkpoint['model_state_dict'])
print("Saved the model to", model_save_path)

# Testing
print("\nStarting final testing...")
model.eval()
test_loss = 0
test_preds = []
test_labels = []

with torch.no_grad():
    for images, batch_labels in tqdm(test_loader, desc="Testing", leave=False):
        images = images.to(device)
        batch_labels = batch_labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, batch_labels)
        test_loss += loss.item()

        _, batch_preds = torch.max(outputs.data, 1)
        test_preds.extend(batch_preds.cpu().numpy())
        test_labels.extend(batch_labels.cpu().numpy())
# Calculating Test Metrics
avg_test_loss = test_loss / len(test_loader)
test_accuracy = accuracy_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds, average='weighted')
test_precision = precision_score(test_labels, test_preds, average='weighted')
test_recall = recall_score(test_labels, test_preds, average='weighted')
# Log Test Metrics
run["test/loss"].log(avg_test_loss)
run["test/accuracy"].log(test_accuracy)
run["test/f1"].log(test_f1)
run["test/precision"].log(test_precision)
run["test/recall"].log(test_recall)
# Print Test Metrics
print("\nFinal Test Metrics:")
print(f"Loss: {avg_test_loss:.4f}")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")

run.stop()


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
<ipython-input-60-763eda777c5c>:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Training epochs:   0%|          | 0/10 [00:00<?, ?it/s]


Starting epoch 1/10



Epoch 1/10:   0%|          | 0/2208 [00:00<?, ?it/s]<ipython-input-60-763eda777c5c>:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():

Epoch 1/10:   0%|          | 1/2208 [00:01<1:11:06,  1.93s/it]<ipython-input-60-763eda777c5c>:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():

Epoch 1/10:   0%|          | 3/2208 [00:02<19:50,  1.85it/s]  

Batch 0: Training loss = 6.0214, Training accuracy = 0.0078, F1 = 0.0104



Epoch 1/10:   1%|          | 26/2208 [00:03<02:19, 15.65it/s]<ipython-input-60-763eda777c5c>:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():

Epoch 1/10:   1%|▏         | 29/2208 [00:03<02:05, 17.32it/s]

...

Training epochs: 100%|██████████| 10/10 [27:00<00:00, 162.08s/it]


Saving the model...
Model saved to /content/drive/MyDrive/TKPR221/models/ResNet_Synthetic20241228_141207.pth


<ipython-input-60-763eda777c5c>:130: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_save_path)


Saved the model to /content/drive/MyDrive/TKPR221/models/ResNet_Synthetic20241228_141207.pth

Starting final testing...



Final Test Metrics:
Loss: 0.1302
Accuracy: 0.9607
F1 Score: 0.9606
Precision: 0.9620
Recall: 0.9607
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 5 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 5 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/muguryalcin/TKPR221/e/TKPR-193/metadata


In [ ]:
# Calculate and print class-wise F1 scores
class_f1 = f1_score(test_labels, test_preds, average=None)

print("\nF1 Scores by Class:")
for class_idx, f1 in enumerate(class_f1):
    print(f"Class {class_idx}: {f1:.4f}")

# Create a more detailed classification report
from sklearn.metrics import classification_report
class_report = classification_report(test_labels, test_preds)
print("\nDetailed Classification Report:")
print(class_report)



F1 Scores by Class:
Class 0: 0.9217
Class 1: 0.7880
Class 2: 0.7165
Class 3: 0.7206
Class 4: 0.7280
Class 5: 0.8721
Class 6: 0.7201
Class 7: 0.6655
Class 8: 0.8692
Class 9: 0.9622
Class 10: 0.5463
Class 11: 0.7204
Class 12: 0.8230
Class 13: 0.7396
Class 14: 0.7545
Class 15: 0.9854
Class 16: 0.8436
Class 17: 0.8282
Class 18: 0.7140
Class 19: 0.9079
Class 20: 0.8787
Class 21: 0.9041
Class 22: 0.8421
Class 23: 0.6740
Class 24: 0.7706
Class 25: 0.6443
Class 26: 0.5146
Class 27: 0.7025
Class 28: 0.7763
Class 29: 0.7735
Class 30: 0.9076
Class 31: 0.8539
Class 32: 0.7120
Class 33: 0.7628
Class 34: 0.8956
Class 35: 0.8274
Class 36: 0.9734
Class 37: 0.9854
Class 38: 0.9073
Class 39: 0.9133
Class 40: 0.9535
Class 41: 0.9264
Class 42: 0.9045
Class 43: 0.9754
Class 44: 0.9667
Class 45: 0.9042
Class 46: 0.8787
Class 47: 0.9938
Class 48: 0.9664
Class 49: 0.7562
Class 50: 0.9140
Class 51: 0.9732
Class 52: 0.9658
Class 53: 0.9959
Class 54: 0.9896
Class 55: 0.9053
Class 56: 0.9412
Class 57: 0.9388
Cla

## Using CNN Model with SGD Classifier

IF YOU WANT TO TRAIN JUST CNN, DON'T USE THIS PART

This part uses already trained CNN to extract features and feed them into the SGD model. You can use the CNN that you have trained in the first part (if so, dont change the model path), or you can use already trained model (change the model path to the path of the trained model from Google Drive).

In [ ]:
# Load the CNN model
convolution_model = CNN(n_classes,return_convolution=True)
model_path = f'/content/drive/MyDrive/TKPR221/models/{run_name}.pth' # CHANGE THE MODEL NAME ACCORDINGLY FROM DRIVE (EACH MODEL HAS UNIQUE NAME)
convolution_model.load_state_dict(torch.load(model_path)['model_state_dict'])
sgd_clf = SGDClassifier(
    loss='hinge',
    penalty='elasticnet',
    max_iter=1000,
)


<ipython-input-49-ce5802ff562c>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  convolution_model.load_state_dict(torch.load(model_path)['model_state_dict'])


Initializing new run for the Convolutional SGD model

In [ ]:
# Initialize Neptune for experiment tracking
run_name = f"ConvolutionalML_{time.strftime('%Y%m%d_%H%M%S')}"
run = neptune.init_run(
    # DON'T CHANGE THIS
    project="muguryalcin/TKPR221",
    # ENTER YOUR API TOKEN HERE
    api_token="",
    # Name of the experiment using timestamp to prevent confusion
    name=run_name,
    # Description of the experiment
    description="Using pretrained CNN to extract features and feed them to SGD for traffic sign classification",
    # Tags describing the experiment
    tags=["C+ML","SGDClassifier"]
)

# Log the hyperparameters
run["hyperparameters"] = {
    "n_epochs": n_epochs,
    # Classifier Specific Parameters
    "model": {
        "type": "SGDClassifier",
        "loss": sgd_clf.get_params()['loss'],
        "penalty": sgd_clf.get_params()['penalty'],
        "alpha": sgd_clf.get_params()['alpha'],
        "max_iter": sgd_clf.get_params()['max_iter']
    },
    # Dataset Specific Parameters
    "dataset": {
        "batch_size": batch_size,
        "total_size": len(train_dataset),
        "train_size": len(train_df),
        "val_size": len(val_df),
        "test_size": len(test_df),
        "n_classes": len(train_dataset.label_encoder.classes_),
        "image_size": image_size,
        "flattened": True,
        "transformations": str(train_transform)  # Convert transform to string representation
    },
}

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/muguryalcin/TKPR221/e/TKPR-191


Train Loop

In [ ]:
# Get unique classes and encode them
unique_classes = train_dataset.label_encoder.classes_

for epoch in tqdm(range(n_epochs), desc="Training epochs"):
    train_preds = []
    train_labels = []
    print(f"\nStarting epoch {epoch+1}/{n_epochs}")

    for i, (images, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}", leave=False)):
        # Get CNN features
        with torch.no_grad():
            images = images.to(device)
            features = model(images).cpu().numpy()
        labels = labels.numpy()
        # Train SGD on batch
        sgd_clf.partial_fit(features, labels, classes=unique_classes)
        preds = sgd_clf.predict(features)
        # Store predictions and labels for metrics
        train_preds.extend(preds)
        train_labels.extend(labels)

        if i % 25 == 0:
            batch_acc = accuracy_score(labels, preds)
            batch_f1 = f1_score(labels, preds, average='weighted')
            batch_precision = precision_score(labels, preds, average='weighted')
            batch_recall = recall_score(labels, preds, average='weighted')
            print(f"Batch {i}: Training accuracy = {batch_acc:.4f}, F1 = {batch_f1:.4f}, Precision = {batch_precision:.4f}, Recall = {batch_recall:.4f}")

    # Calculate Train Metrics
    train_accuracy = accuracy_score(train_labels, train_preds)
    train_f1 = f1_score(train_labels, train_preds, average='weighted')
    train_precision = precision_score(train_labels, train_preds, average='weighted')
    train_recall = recall_score(train_labels, train_preds, average='weighted')
    # Log Train Metrics
    run["train/accuracy"].log(train_accuracy)
    run["train/f1"].log(train_f1)
    run["train/precision"].log(train_precision)
    run["train/recall"].log(train_recall)
    # Print Train Metrics
    print(f"\nEpoch {epoch+1} training accuracy: {train_accuracy:.2f}%")
    print(f"Epoch {epoch+1} training F1 score: {train_f1:.4f}")
    print(f"Epoch {epoch+1} training precision: {train_precision:.4f}")
    print(f"Epoch {epoch+1} training recall: {train_recall:.4f}")

    # Validation
    print("\nStarting validation...")
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation", leave=False):
            images = images.to(device)
            features = model(images).cpu().numpy()
            preds = sgd_clf.predict(features)
            val_preds.extend(preds)
            val_labels.extend(labels.numpy())

    # Calculate Validation Metrics
    val_accuracy = accuracy_score(val_labels, val_preds)
    val_f1 = f1_score(val_labels, val_preds, average='weighted')
    val_precision = precision_score(val_labels, val_preds, average='weighted')
    val_recall = recall_score(val_labels, val_preds, average='weighted')
    # Log Validation Metrics
    run["validation/accuracy"].log(val_accuracy)
    run["validation/f1"].log(val_f1)
    run["validation/precision"].log(val_precision)
    run["validation/recall"].log(val_recall)
    # Print Validation Metrics
    print(f"Validation accuracy: {val_accuracy:.2f}%")
    print(f"Validation F1 score: {val_f1:.4f}")
    print(f"Validation precision: {val_precision:.4f}")
    print(f"Validation recall: {val_recall:.4f}")

# TEST
print("\nStarting testing...")
test_preds = []
test_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing", leave=False):
        images = images.to(device)
        features = model(images).cpu().numpy()
        preds = sgd_clf.predict(features)
        test_preds.extend(preds)
        test_labels.extend(labels.numpy())

# Calculate Test Metrics
test_accuracy = accuracy_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds, average='weighted')
test_precision = precision_score(test_labels, test_preds, average='weighted')
test_recall = recall_score(test_labels, test_preds, average='weighted')
# Log Test Metrics
run["test/accuracy"].log(test_accuracy)
run["test/f1"].log(test_f1)
run["test/precision"].log(test_precision)
run["test/recall"].log(test_recall)
# Print Test Metrics
print(f"Test accuracy: {test_accuracy:.2f}%")
print(f"Test F1 score: {test_f1:.4f}")
print(f"Test precision: {test_precision:.4f}")
print(f"Test recall: {test_recall:.4f}")

run.stop()

Training epochs:   0%|          | 0/5 [00:00<?, ?it/s]


Starting epoch 1/5



Epoch 1/5:   0%|          | 0/2208 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

Epoch 1/5:   0%|          | 1/2208 [00:01<1:07:19,  1.83s/it]

Batch 0: Training accuracy = 0.0625, F1 = 0.0456, Precision = 0.0448, Recall = 0.0625



Epoch 1/5:   1%|          | 25/2208 [00:10<13:20,  2.73it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

Epoch 1/5:   1%|          | 26/2208 [00:10<13:52,  2.62it/s]

...


Epoch 5/5: 100%|██████████| 2208/2208 [13:16<00:00,  2.95it/s]
                                                              


Epoch 5 training accuracy: 0.87%
Epoch 5 training F1 score: 0.8763
Epoch 5 training precision: 0.8891
Epoch 5 training recall: 0.8687

Starting validation...



Validation: 100%|█████████▉| 551/552 [00:45<00:00, 17.22it/s]
                                                             /usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Training epochs: 100%|██████████| 5/5 [1:09:49<00:00, 837.85s/it]


Validation accuracy: 0.76%
Validation F1 score: 0.7667
Validation precision: 0.8320
Validation recall: 0.7563

Starting testing...


Test accuracy: 0.76%
Test F1 score: 0.7679
Test precision: 0.8331
Test recall: 0.7580
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 4 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 4 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/muguryalcin/TKPR221/e/TKPR-191/metadata


Saving the Model

In [ ]:
from joblib import dump, load
print("Saving the model...")
model_save_path = f'/content/drive/MyDrive/TKPR221/models/{run_name}.joblib'
dump(sgd_clf, model_save_path)
print("Saved the model to", model_save_path)

# References
Throughout this notebook, AI tools such as ChatGPT are used used for mainly for debugging purposes. However some of the parts are edited as well, but the code is mainly written by us.